# Day 16

In [1]:
from ortools.graph import pywrapgraph

## Part 1

In [2]:
def parse_line(line:str)->(list[str],int,list[str]):
    from_node = line.split(' has ')[0].split('Valve ')[1]
    flow = int(line.split('rate=')[1].split(';')[0])
    to_nodes = line.split('valve')[1].split('\n')[0].strip('s').replace(" ", "").split(',')
    return [from_node]*len(to_nodes), [flow]*len(to_nodes), to_nodes

In [3]:
with open('test_day16.txt') as input_text:
    from_nodes = []
    to_nodes = []
    weights = []
    for line in input_text:
        n1,w,n2 = parse_line(line)
        from_nodes.extend(n1)
        to_nodes.extend(n2)
        weights.extend(w)

In [12]:
z = list(set(from_nodes).union(set(to_nodes)))
node_mapping = dict(zip(z,range(len(z))))

In [14]:
from_node_indexes = [node_mapping[n] for n in from_nodes]
to_node_indexes = [node_mapping[n] for n in to_nodes]

In [15]:
max_flow = pywrapgraph.SimpleMaxFlow()
for i in range(0,len(from_nodes)):
    max_flow.AddArcWithCapacity(from_node_indexes[i], to_node_indexes[i], weights[i])


## Part 2

In [25]:
import collections
import queue
import heapq
import cProfile

def parse_input(filename):
    valves, tunnels = {}, {}
    for line in open(filename).read().split('\n'):
        valve = line[6:8]
        flow_rate = int(line.split('=')[1].split(';')[0])
        new_tunnels = [x.strip()
                       for x in line.split('valve')[1][1:].split(',')]
        valves[valve] = flow_rate
        tunnels[valve] = new_tunnels
    return [valves, tunnels]

def calc_distances(valves, tunnels):
    """ Floyd-Warshall FTW """
    dist = {a: {b: 1000 for b in valves} for a in valves}
    for u in valves:
        dist[u][u] = 0
        for v in tunnels[u]:
            dist[u][v] = 1
    for k in valves:
        for i in valves:
            for j in valves:
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    return dist

State = collections.namedtuple("State", "flow,p1,t1,p2,t2,closed")
State.time = lambda s: min(s.t1, s.t2)

def generate_new_human_states(valves, dist, s):
    if s.time() != s.t1:
        return [s]

    if valves[s.p1] and s.p1 in s.closed:
        return [s._replace(
            flow=s.flow - (29 - s.time()) * valves[s.p1],
            t1=s.t1 + 1,
            closed=s.closed - {s.p1})]

    new_states = [s._replace(p1=dest, t1=s.t1 + dist[s.p1][dest])
                  for dest in s.closed]

    if not new_states:
        return [s._replace(t1=30)]
    else:
        return new_states

def generate_new_elephant_states(valves, dist, states):
    new_states = []
    for s in states:
        if s.time() != s.t2:
            new_states.append(s)
        elif valves[s.p2] and s.p2 in s.closed:
            new_states.append(s._replace(
                flow=s.flow - (29 - s.time()) * valves[s.p2],
                t2=s.t2 + 1,
                closed=s.closed - {s.p2}))
        else:
            new_states.extend(
                [s._replace(p2=dest, t2=s.t2 + dist[s.p2][dest]) for dest in s.closed])

    if not new_states:
        return [s._replace(t2=30) for s in states]
    else:
        return new_states

def part1(valves, dist):
    pq = []
    heapq.heappush(pq, State(0, 'AA', 0, 'AA', 30, {
                   k for k, v in valves.items() if v}))

    best, best_for_time = 0, collections.defaultdict(int)
    i = 0
    while pq:
        cur = heapq.heappop(pq)
        best = min(cur.flow, best)
        best_for_time[cur.time()] = min(best_for_time[cur.time()], cur.flow)
        if cur.time() < 30 and cur.closed and (cur.time() < 10 or 1.5 * cur.flow <= best_for_time[cur.time()]):
            for s in generate_new_human_states(valves, dist, cur):
                heapq.heappush(pq, s)

    print(-best)

def part2(valves, dist):
    pq = []
    heapq.heappush(pq, State(0, 'AA', 4, 'AA', 4, set(
        k for k, v in valves.items() if v)))
    best, best_for_time = 0, collections.defaultdict(int)
    while pq:
        cur = heapq.heappop(pq)
        best = min(cur.flow, best)
        best_for_time[cur.time()] = min(best_for_time[cur.time()], cur.flow)
        if cur.time() < 30 and cur.closed:
            if cur.time() < 12 or 1.25 * cur.flow <= best_for_time[cur.time()]:
                h_states = generate_new_human_states(valves, dist, cur)
                e_states = generate_new_elephant_states(valves, dist, h_states)
                for s in e_states:
                    heapq.heappush(pq, s)
    print(-best)

valves, tunnels = parse_input('input_day16.txt')
dist = calc_distances(valves, tunnels)
#part1(valves, dist)
part2(valves, dist)


2904


In [26]:
#!/usr/bin/env python3

import re
from collections import deque

DAY_NUM = 16
DAY_DESC = 'Day 16: Proboscidea Volcanium'

class Node:
    __slots__ = ['rate', 'node_name', 'tunnels', 'opened', 'valve_num']
    def __init__(self, value, next_valve_num):
        m = re.search("Valve (.*?) has flow rate=(.*?)\\; tunnels? leads? to valves? (.*?)$", value)
        self.node_name, self.rate, self.tunnels = m.groups()
        self.rate = int(self.rate)
        self.tunnels = self.tunnels.split(", ")
        self.opened = False
        if self.rate > 0:
            self.valve_num = 1 << next_valve_num[0]
            next_valve_num[0] += 1
        else:
            self.valve_num = 0
    
class State:
    __slots__ = ['a', 'b', 'opened', 'pressure']
    def __init__(self, a, b, opened, pressure):
        self.a = a
        self.b = b
        self.opened = opened
        self.pressure = pressure

    def __repr__(self):
        return f"{self.a}-{self.b},{self.opened},{self.pressure}"

def calc(log, values, mode):
    next_valve_num = [0]
    nodes = {x.node_name: x for x in [Node(y, next_valve_num) for y in values]}
    first = min(nodes.keys())

    best = {}
    target_best = 0

    def is_best(val):
        if val.pressure < target_best:
            return False

        key = (val.a, val.b, val.opened)

        if key not in best or val.pressure > best[key]:
            best[key] = val.pressure
            return True
            
        return False

    best_history = []
    total_time = 30 if mode == 1 else 26
    states = deque([State(first, None if mode == 1 else first, 0, 0)])

    for time in range(1, total_time + 1):
        best_history.append(0 if len(best) == 0 else max(best.values()))
        best_change = max(x.rate for x in nodes.values()) * (total_time - time)
        target_best = int(best_history[-1] - best_change)
        if mode == 2:
            if len(best_history) > 3 and sum(best_history[-3:]) == best_history[-1] * 3:
                return best_history[-1]

        next_states = deque()
        for cur in states:
            node_a = nodes[cur.a]
            node_b = None if cur.b is None else nodes[cur.b]

            if node_a.rate > 0 and (node_a.valve_num & cur.opened) == 0:
                if node_b is not None and node_b.rate > 0 and (node_b.valve_num & cur.opened) == 0 and node_b.valve_num != node_a.valve_num:
                    val = State(
                        node_a.node_name, 
                        node_b.node_name, 
                        cur.opened | node_a.valve_num | node_b.valve_num, 
                        cur.pressure + node_a.rate * (total_time - time) + node_b.rate * (total_time - time),
                    )
                    if is_best(val): next_states.append(val)
                for y in [None] if node_b is None else node_b.tunnels:
                    val = State(node_a.node_name, y, cur.opened | node_a.valve_num, cur.pressure + node_a.rate * (total_time - time))
                    if is_best(val): next_states.append(val)
            for x in node_a.tunnels:
                if node_b is not None and node_b.rate > 0 and (node_b.valve_num & cur.opened) == 0:
                    val = State(x, node_b.node_name, cur.opened | node_b.valve_num, cur.pressure + node_b.rate * (total_time - time))
                    if is_best(val): next_states.append(val)
                for y in [None] if node_b is None else node_b.tunnels:
                    val = State(x, y, cur.opened, cur.pressure)
                    if is_best(val): next_states.append(val)
        states = next_states

    return best_history[-1]

def test(log):
    values = log.decode_values("""
        Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
        Valve BB has flow rate=13; tunnels lead to valves CC, AA
        Valve CC has flow rate=2; tunnels lead to valves DD, BB
        Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
        Valve EE has flow rate=3; tunnels lead to valves FF, DD
        Valve FF has flow rate=0; tunnels lead to valves EE, GG
        Valve GG has flow rate=0; tunnels lead to valves FF, HH
        Valve HH has flow rate=22; tunnel leads to valve GG
        Valve II has flow rate=0; tunnels lead to valves AA, JJ
        Valve JJ has flow rate=21; tunnel leads to valve II
    """)

    log.test(calc(log, values, 1), 1651)
    log.test(calc(log, values, 2), 1707)

def run(log, values):
    log(calc(log, values, 1))
    log(calc(log, values, 2))



In [27]:
with open("input_day16.txt") as f: values = [x.strip("\r\n") for x in f.readlines()]
print(f"Running day {DAY_DESC}:")
run(print, values)

Running day Day 16: Proboscidea Volcanium:
2183
2905


In [28]:
from time import time
import networkx as nx
import matplotlib.pyplot as plt
from copy import copy
from itertools import combinations
 
 
def timer_func(func):
    # This function shows the execution time of
    # the function object passed
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {(t2 - t1):.4f}s')
        return result
 
    return wrap_func
 
 
class Cave:
    def __init__(self, name, flow, connected):
        self.name = name
        self.flow = flow
        self.connected = connected
        self.valve_connect = {}  # path length to caves with valves that can flow
 
    def __str__(self):
        return f'Cave {self.name}'
 
    def __repr__(self):
        return f'Cave {self.name}'
 
 
def cave_search(start, path, time_rem, flow, valid_paths):
    for cave in start.valve_connect:
        if cave in path or cave == start:
            continue
        elif time_rem - start.valve_connect[cave] <= 0:
            continue
        else:
            new_time_rem = time_rem - start.valve_connect[cave]
            new_flow = flow + new_time_rem * cave.flow
            valid_paths[path + tuple([cave])] = {'time': new_time_rem, 'flow': new_flow}
            valid_paths = cave_search(cave, path + tuple([cave]), new_time_rem, new_flow, valid_paths)
    return valid_paths
 
@timer_func
def day16(filepath, part2=False):
    with open(filepath) as fin:
        lines = fin.readlines()
 
    caves = []
    closed_valves = []
 
    for line in lines:
        cave = line.split()[1]
        flow_rate = int(line.split('=')[-1].split(';')[0])
        if 'valves' in line:
            connected_caves = [entry.strip() for entry in line.split('lead to valves')[-1].split(',')]
        else:
            connected_caves = [line.split('leads to valve')[-1].strip()]
        caves.append(Cave(cave, flow_rate, connected_caves))
        if caves[-1].flow > 0:
            closed_valves.append(caves[-1])
        if caves[-1].name == 'AA':
            start = caves[-1]
 
    G = nx.Graph()
    for cave in caves:
        for c_cave in cave.connected:
            for cave1 in caves:
                if cave1.name == c_cave:
                    v = cave1
                    break
            G.add_edge(cave, v)
 
    for cave in closed_valves:
        start.valve_connect[cave] = len(nx.shortest_path(G, start, cave))
        for cave1 in closed_valves:
            if cave1 != cave:
                cave.valve_connect[cave1] = len(nx.shortest_path(G, cave, cave1))
 
    valid_paths = cave_search(start, (), 30, 0, {})
 
    max_flow = max([val['flow'] for val in valid_paths.values()])
    print(f'Part 1: {max_flow}')
 
    closed_valves_len = len(closed_valves)
    new_valid_paths = {}
    for key, value in valid_paths.items():
        if value['time'] > 4:
            new_valid_paths[key] = value
    flows = []
    for a, b in combinations(new_valid_paths, 2):
        if len(set(a).intersection(b)) > 0:
            continue
        else:
            time_rem = 26 - start.valve_connect[a[0]]
            a_flow = a[0].flow*time_rem
            if len(a) > 1:
                for i, entry in enumerate(a[1:]):
                    time_rem = time_rem - a[i].valve_connect[entry]
                    a_flow += time_rem * entry.flow
            time_rem = 26 - start.valve_connect[b[0]]
            b_flow = b[0].flow * time_rem
            if len(b) > 1:
                for i, entry in enumerate(b[1:]):
                    time_rem = time_rem - b[i].valve_connect[entry]
                    b_flow += time_rem * entry.flow
 
            flows.append(a_flow + b_flow)
 
    max_flow = max(flows)
    print(f'Part 2: {max_flow}')

In [30]:
day16('test_day16.txt')

Part 1: 1651
Part 2: 1707
Function 'day16' executed in 0.4516s


In [31]:
day16('input_day16.txt')

Part 1: 2183
Part 2: 2911
Function 'day16' executed in 412.2584s
